# ESA Project: Fake or Real: The Impostor Hunt in Texts

This notebook is dedicated to **model training**.  
It covers:

- Load the pre-tokenized and saved PyTorch datasets (`tokenized_train.pt` and `tokenized_val.pt`) created in the data creation step.
- Instantiate the pre-trained **DistilBertForSequenceClassification** model, configured for a binary classification task (Real or fake).
- Define the **TrainingArguments** for the Hugging Face Trainer, including hyperparameters such as batch size, learning rate, number of epochs, and logging settings.
- Implemente a function to compute and track key performance metrics on the validation set including **Accuracy, Precision, Recall and F1-score**.
- Executing the training loop using the Hugging Face **Trainer** class which manages the entire process including checkpointing and model saving.
- Save the best-performing model and its tokenizer to disk for later use in inference and deployment.

# Import librairies

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import re

import torch
from torch.nn import CrossEntropyLoss
from transformers import (
    DistilBertForSequenceClassification, 
    Trainer, 
    TrainingArguments, 
    AutoTokenizer,
    TrainerCallback, 
    TrainerState, 
    TrainerControl,
    EarlyStoppingCallback
)
from transformers import AutoConfig, AutoModelForSequenceClassification

from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score, 
    precision_recall_fscore_support, 
    confusion_matrix
)

import os
import sys

# Add the src folder to Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
import config
from preprocessing import TextPreprocessor, get_text_statistics

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/photoli93/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Create Custom Dataset Class

In [2]:
class TextPairDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# Load tokenized datasets

In [3]:
TOKENIZED_TRAIN_PATH = config.PROCESSED_DATA_DIR / "tokenized_train.pt"
TOKENIZED_VAL_PATH = config.PROCESSED_DATA_DIR / "tokenized_val.pt"

OUTPUT_DIR = config.OUTPUT_DIR / "distilbert_fake_or_real"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Load tokenized datasets
print(f"Loading tokenized training data from: {TOKENIZED_TRAIN_PATH}")
try:
    train_dataset = torch.load(TOKENIZED_TRAIN_PATH, weights_only=False)
    val_dataset = torch.load(TOKENIZED_VAL_PATH, weights_only=False)
except FileNotFoundError:
    print("Error: Tokenized data not found. Please ensure 03_dataset_creation.py (or your data creation notebook) was run successfully")
    sys.exit(1)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Loading tokenized training data from: /Users/photoli93/Desktop/Projets perso Python/esa_fake_or_real/data/processed/tokenized_train.pt
Training dataset size: 212
Validation dataset size: 41


# Load model and tokenizer

In [4]:
# Load the model for sequence classification with 2 labels (Real/Fake)
print(f"Loading model: {config.TOKENIZER_NAME} for Sequence Classification")
model = DistilBertForSequenceClassification.from_pretrained(
    config.TOKENIZER_NAME, 
    num_labels=2
)

# Might need the tokenizer later for prediction/evaluation
tokenizer = AutoTokenizer.from_pretrained(config.TOKENIZER_NAME)

Loading model: distilbert-base-uncased for Sequence Classification


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Define parameters and metrics

In [5]:
# Detect device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# Training configuration
training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),          
    num_train_epochs=20,                 # 20 epochs but with early stopping to avoid overfitting
    per_device_train_batch_size=16,      
    per_device_eval_batch_size=64,       
    warmup_steps=500,                    # Nb of steps during which the learning rate increases learning
    weight_decay=0.01,                   # Regularization term to reduce overfitting by penalizing large weights
    logging_dir='./logs',                
    logging_steps=50,                    # Log metrics every 50 training steps
    evaluation_strategy="epoch",         # Evaluate at the end of each epoch
    save_strategy="epoch",               # Save model checkpoint at the end of each epoch
    save_total_limit=2,                  # Keep only 2 best checkpoints
    load_best_model_at_end=True,         # Load the best model found during training
    metric_for_best_model="eval_f1",     # Metric to monitor for best model
    greater_is_better=True,              # Looking to maximize the f1 metric
    fp16=False,                          # Disable for Mac, Transformers will automatically use MPS
    report_to="none"                     # Disables wandb, tensorboard, etc. (disabling sending logs online)
)

# Define Evaluation Metric
try:
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    
    # p is an object passed by the Hugging Face Trainer. It contains "predictions" and "label_ids"
    def compute_metrics(p):
        preds = np.argmax(p.predictions, axis=1)
        precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
        acc = accuracy_score(p.label_ids, preds)
        return {
            'accuracy': acc,
            'f1': f1,
            'precision': precision,
            'recall': recall
        }
except ImportError:
    print("Warning: scikit-learn not found. Install with 'pip install scikit-learn' to use advanced metrics")
    def compute_metrics(p):
        return {}

Using device: mps


# Custom Loss function

By exploding chunks, it has added unbalance in train dataset (138 for class 0 and 74 for class 1) so custom loss function has to be defined in order to add weights on classes

In [6]:
train_df = pd.read_csv(config.PROCESSED_DATA_DIR / "train_exploded.csv")

# Count of each class
counts = train_df['label'].value_counts().sort_index()
print(counts)

# Dynamic class weights (inverse frequency)
weights = 1.0 / counts
weights = weights / weights.sum()  # Normalize
weights = torch.tensor(weights.values, dtype=torch.float)
print(weights)

# Override the default loss
def compute_loss(model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fct = CrossEntropyLoss(weight=weights.to(logits.device))
    loss = loss_fct(logits, labels)
    return (loss, outputs) if return_outputs else loss

label
0    138
1     74
Name: count, dtype: int64
tensor([0.3491, 0.6509])


# Custom subclass Trainer

Since in Hugging Face Transformers, the **Trainer** class does not have a `compute_loss` argument in its constructor, I created a subclass called **WeightedTrainer** to use the custom `compute_loss` method.

In [7]:
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Define class weights
        weights = torch.tensor([1.0, 138/74]).to(logits.device)
        loss_fct = CrossEntropyLoss(weight=weights)
        
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Custom Confusion matrix callback

In [ ]:
class ConfusionMatrixCallback(TrainerCallback):
    def __init__(self):
        self.trainer = None
        self.best_epoch = 0
        self.best_f1 = -float('inf')
    
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """Called after evaluation"""
        if not hasattr(self, 'trainer') or self.trainer is None:
            print("Trainer not set in ConfusionMatrixCallback")
            return
        
        try:
            # Get predictions
            preds_output = self.trainer.predict(self.trainer.eval_dataset)
            preds = np.argmax(preds_output.predictions, axis=1)
            labels = preds_output.label_ids
            
            # Compute confusion matrix
            cm = confusion_matrix(labels, preds)
            
            # Get current metrics
            current_epoch = int(state.epoch) if state.epoch else 0
            current_f1 = metrics.get('eval_f1', 0) if metrics else 0
            
            # Track best
            is_best = False
            if current_f1 > self.best_f1:
                self.best_f1 = current_f1
                self.best_epoch = current_epoch
                is_best = True
            
            # Print header
            print(f"\n{'='*80}")
            if is_best:
                print(f"[Epoch {current_epoch}] Confusion Matrix (New best)")
            else:
                print(f"[Epoch {current_epoch}] Confusion Matrix")
            print(f"{'='*80}")
            
            # Print confusion matrix with labels
            print("Confusion Matrix:")
            print("              Predicted")
            print("           Real    Fake")
            print(f"Real   [[{cm[0][0]:4d}    {cm[0][1]:4d}]]")
            print(f"Fake   [[{cm[1][0]:4d}    {cm[1][1]:4d}]]")
            
            # Calculate per-class metrics
            tn, fp, fn, tp = cm.ravel()
            
            # Class metrics
            real_precision = tn / (tn + fn) if (tn + fn) > 0 else 0
            real_recall = tn / (tn + fp) if (tn + fp) > 0 else 0
            fake_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            fake_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            
            print(f"\nPer-Class Metrics:")
            print(f"  Real: Precision={real_precision:.3f}, Recall={real_recall:.3f}")
            print(f"  Fake: Precision={fake_precision:.3f}, Recall={fake_recall:.3f}")
            
            # Overall metrics
            if metrics:
                print(f"\nOverall Metrics:")
                print(f"  Accuracy:  {metrics.get('eval_accuracy', 0):.4f}")
                print(f"  F1:        {metrics.get('eval_f1', 0):.4f}")
                print(f"  Precision: {metrics.get('eval_precision', 0):.4f}")
                print(f"  Recall:    {metrics.get('eval_recall', 0):.4f}")
                print(f"  Loss:      {metrics.get('eval_loss', 0):.4f}")
            
            print(f"\nBest so far: Epoch {self.best_epoch}, F1={self.best_f1:.4f}")
            print(f"{'='*80}\n")
            
        except Exception as e:
            print(f"Error in ConfusionMatrixCallback: {e}")
    
    def on_train_end(self, args, state, control, **kwargs):
        """Verify best model was loaded"""
        print(f"\n{'='*80}")
        print(f"TRAINING COMPLETE")
        print(f"{'='*80}")
        print(f"Best Epoch: {self.best_epoch}")
        print(f"Best F1: {self.best_f1:.4f}")
        
        if state.best_model_checkpoint:
            print(f"Best checkpoint: {state.best_model_checkpoint}")
            from pathlib import Path
            if Path(state.best_model_checkpoint).exists():
                print(f"Checkpoint verified on disk")
            else:
                print(f"Checkpoint NOT found")
        else:
            print(f"No best checkpoint saved")
        
        # Final verification
        if hasattr(self, 'trainer') and self.trainer is not None:
            print(f"\nVerifying loaded model")
            try:
                preds_output = self.trainer.predict(self.trainer.eval_dataset)
                preds = np.argmax(preds_output.predictions, axis=1)
                labels = preds_output.label_ids
                
                from sklearn.metrics import f1_score
                final_f1 = f1_score(labels, preds, average='binary')
                
                print(f"Final Model F1: {final_f1:.4f}")
                print(f"Expected Best F1: {self.best_f1:.4f}")
                
                if abs(final_f1 - self.best_f1) < 0.001:
                    print(f"Correct: Best model was loaded")
                else:
                    print(f"Error: Wrong model! Expected {self.best_f1:.4f}, got {final_f1:.4f}")
            except Exception as e:
                print(f"Could not verify: {e}")
        
        print(f"{'='*80}\n")
    
    def set_trainer(self, trainer):
        self.trainer = trainer

# Training process

In [9]:
# Cross-validation settings
k = 4
kf = KFold(n_splits=k, shuffle=True, random_state=42)

all_fold_metrics = []

for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
    print("-" * 80)
    print(f"\nFold {fold+1}/{k}")
    print("-" * 80)
    
    # Subsets for this fold
    train_fold = torch.utils.data.Subset(train_dataset, train_idx)
    val_fold = torch.utils.data.Subset(train_dataset, val_idx)
    
    # Load pre-trained model
    model_fold = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=2
    )
    
    # Callback for the confusion matrix
    cm_callback_fold = ConfusionMatrixCallback()
    
    # Init trainer for this fold
    trainer_fold = WeightedTrainer(
        model=model_fold,
        args=training_args,
        train_dataset=train_fold,
        eval_dataset=val_fold,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3),
                   cm_callback_fold]
    )
    
    cm_callback_fold.set_trainer(trainer_fold)
    
    # Training
    trainer_fold.train()

    # Print fold state for clarity
    print("-" * 80)
    print("\nFold trainer state")
    print(f"Best checkpoint: {trainer_fold.state.best_model_checkpoint}")
    print(f"Best metric value ({training_args.metric_for_best_model}): {trainer_fold.state.best_metric}")
    print("-" * 80)

    # Save best model of this fold (will save the reloaded best model)
    fold_model_path = OUTPUT_DIR / f"fold_{fold+1}"
    trainer_fold.save_model(str(fold_model_path))
    
    # Evaluation (will use the reloaded best model weights)
    metrics = trainer_fold.evaluate()
    all_fold_metrics.append(metrics)
    print(f"Fold {fold+1} evaluation (Best model metrics)")
    print(metrics)

    # Memory cleaning
    del model_fold, trainer_fold
    torch.mps.empty_cache()

# Average metrics on all the folds
avg_metrics = {k: np.mean([m[k] for m in all_fold_metrics if k in m]) for k in all_fold_metrics[0]}
print("\nCross-validation summary")
print(avg_metrics)

# Final training on full dataset
print("\nFinal training on full dataset")

final_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

cm_callback_final = ConfusionMatrixCallback()
final_trainer = WeightedTrainer(
    model=final_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3),
               cm_callback_final]
)
cm_callback_final.set_trainer(final_trainer)

print("\nStarting final training on full training dataset")
final_trainer.train()

# The best model is automatically loaded into the trainer's model attribute
# because load_best_model_at_end=True was set in TrainingArguments.
# We save the model that is currently loaded in the trainer.

# Print final state for clarity
print("-" * 80)
print("\nFinal trainer state")
print(f"Best checkpoint: {final_trainer.state.best_model_checkpoint}")
print(f"Best metric value ({training_args.metric_for_best_model}): {final_trainer.state.best_metric}")
print("-" * 80)

# Final evaluation (will use the reloaded best model weights)
final_metrics = final_trainer.evaluate()

# Final save (will save the reloaded best model)
final_model_path = OUTPUT_DIR / "final_model"
final_trainer.save_model(str(final_model_path))
tokenizer.save_pretrained(str(final_model_path))
print(f"Final model and tokenizer saved to: {final_model_path}")


--------------------------------------------------------------------------------

Fold 1/4
--------------------------------------------------------------------------------


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.691260814666748, 'eval_accuracy': 0.6792452830188679, 'eval_f1': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 2.2064, 'eval_samples_per_second': 24.021, 'eval_steps_per_second': 0.453, 'epoch': 1.0}

[Epoch 1] Confusion Matrix (New best)
Confusion Matrix:
              Predicted
           Real    Fake
Real   [[  36       0]]
Fake   [[  17       0]]

Per-Class Metrics:
  Real: Precision=0.679, Recall=1.000
  Fake: Precision=0.000, Recall=0.000

Overall Metrics:
  Accuracy:  0.6792
  F1:        0.0000
  Precision: 0.0000
  Recall:    0.0000
  Loss:      0.6913

Best So Far: Epoch 1, F1=0.0000



...

{'eval_loss': 0.5848348140716553, 'eval_accuracy': 0.8536585365853658, 'eval_f1': 0.8235294117647058, 'eval_precision': 0.9333333333333333, 'eval_recall': 0.7368421052631579, 'eval_runtime': 3.8268, 'eval_samples_per_second': 10.714, 'eval_steps_per_second': 0.261, 'epoch': 10.0}

[Epoch 10] Confusion Matrix
Confusion Matrix:
              Predicted
           Real    Fake
Real   [[  21       1]]
Fake   [[   5      14]]

Per-Class Metrics:
  Real: Precision=0.808, Recall=0.955
  Fake: Precision=0.933, Recall=0.737

Overall Metrics:
  Accuracy:  0.8537
  F1:        0.8235
  Precision: 0.9333
  Recall:    0.7368
  Loss:      0.5848

Best So Far: Epoch 7, F1=0.8718

{'train_runtime': 319.056, 'train_samples_per_second': 13.289, 'train_steps_per_second': 0.878, 'train_loss': 0.44127853257315497, 'epoch': 10.0}

TRAINING COMPLETE
Best Epoch: 7
Best F1: 0.8718
Best checkpoint: /Users/photoli93/Desktop/Projets perso Python/esa_fake_or_real/results/distilbert_fake_or_real/checkpoint-98
Checkpo

  0%|          | 0/1 [00:00<?, ?it/s]

Final Model F1: 0.8718
Expected Best F1: 0.8718
Correct: Best model was loaded

--------------------------------------------------------------------------------

Final trainer state
Best checkpoint: /Users/photoli93/Desktop/Projets perso Python/esa_fake_or_real/results/distilbert_fake_or_real/checkpoint-98
Best metric value (eval_f1): 0.8717948717948718
--------------------------------------------------------------------------------


  0%|          | 0/1 [00:00<?, ?it/s]


[Epoch 10] Confusion Matrix
Confusion Matrix:
              Predicted
           Real    Fake
Real   [[  19       3]]
Fake   [[   2      17]]

Per-Class Metrics:
  Real: Precision=0.905, Recall=0.864
  Fake: Precision=0.850, Recall=0.895

Overall Metrics:
  Accuracy:  0.8780
  F1:        0.8718
  Precision: 0.8500
  Recall:    0.8947
  Loss:      0.3245

Best So Far: Epoch 7, F1=0.8718

Final model and tokenizer saved to: /Users/photoli93/Desktop/Projets perso Python/esa_fake_or_real/results/distilbert_fake_or_real/final_model


## Observations
### 1. Cross-Validation Summary
The cross-validation results show a pretty good degree of variability in the model's performance across the first 3 folds but vary more across the last fold. This might suggests that the model's performance is sensitive to the specific data split

| Fold   | Best Epoch | Best eval_f1 | Best Checkpoint |
| ------ | ---------- | ------------ | --------------- |
| Fold 1 | 8          | 0.8095       | checkpoint-80   |
| Fold 2 | 8          | 0.8444       | checkpoint-80   |
| Fold 3 | 4          | 0.8636       | checkpoint-40   |
| Fold 4 | 4          | 0.6809       | checkpoint-40   |

<ins>NB:</ins> The average F1-score is approximately 0.7996

### 2. Final Training on Full Dataset
The final training run on the full dataset shows a strong result which aligns with the better-performing folds (folds 2 and 3)

| Epoch | `eval_f1` (Best metric) | `eval_accuracy` | `eval_loss`  | Best so far |
| ----- | --------------------- | ------------- | ---------- | ----------- |
| 1     | 0.0000                | 0.5366        | 0.7050     | Yes       |
| 2     | 0.6500                | 0.6585        | 0.6804     | Yes       |
| 5     | 0.8085                | 0.7805        | 0.4335     | Yes       |
| 6     | 0.8500                | 0.8537        | 0.3556     | Yes       |
| 7     | **0.8718**            | **0.8780**    | **0.3245** | Yes       |
| 8     | 0.8636                | 0.8537        | 0.3006     | No        |
| 9     | 0.8718                | 0.8780        | 0.3251     | No        |
| 10    | 0.8235                | 0.8537        | 0.5848     | No        |

<ins>Best Performance:</ins> The model achieved its peak F1-score of 0.8718 at Epoch 7.
The early stopping makes training terminated after the evaluation of Epoch 10 due to the `early_stopping_patience=3`

<ins>Final saved model:</ins> The final model saved to `/final_model` is confirmed to be the one from Epoch 7 (**checkpoint-98**), with an F1-score of 0.8718

### 3. Class-Specific Performance (Best Model - Epoch 7)
The confusion matrix for the final best model (Epoch 7) shows excellent and balanced performance :
|                 | **Predicted Real** | **Predicted Fake** | **Recall (TPR)**    |
| --------------- | ------------------ | ------------------ | ------------------- |
| **Actual Real** | 19 (True Negative) | 3 (False Positive) | 19 / 22 = **86.4%** |
| **Actual Fake** | 2 (False Negative) | 17 (True Positive) | 17 / 19 = **89.5%** |

The model correctly identifies 17 out of 19 actual Fake texts (Recall: 89.5%)

When the model predicts a text is Real, it is correct 19 out of 21 times (Precision: 90.5%)

## Conclusion
The training run was successful and the final model is robust and well-performing on the validation set. It has strong performance with the final DistilBert model

# End of model training notebook